# Load Input Data

The data that we have on property prices is quite raw, and the numbers in particular are all over the place in terms of scale. To make it easier for the numerical methods in subsequent steps, we first normalise them to a reasonable range.

Also, since addresses are hard to group by proximity, we replace them with geocodes.

We begin by importing required packages.

In [1]:
import pandas as pd
import datetime

Then we load the input data.

In [2]:
postcode_locations = pd.read_csv('../data/NSPL_FEB_2017_UK.csv').set_index('pcds')

/Users/harish/Desktop/uk-property-prices/housing-data/venv/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (30,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
property_transactions = pd.read_csv('../data/pp-complete.csv',
                                        header=None,
                                        names=[
                                            'transaction_id',
                                            'transaction_price',
                                            'transaction_transfer_date',
                                            'property_postcode',
                                            'property_type',
                                            'property_age',
                                            'property_duration',
                                            'property_paon',
                                            'property_saon',
                                            'property_street',
                                            'property_locality',
                                            'property_town_or_city',
                                            'property_district',
                                            'property_county',
                                            'transaction_category',
                                            '_record_type'
                                        ]).fillna('')

We immediately filter the transactions to only include those of type "Standard Price Paid," which includes individual residential properties sold for full market value.

In [4]:
property_transactions = property_transactions.loc[property_transactions['transaction_category']=='A']

In [5]:
property_transactions.shape

(21858755, 16)

Now we construct a new dataframe that contains all the necessary input (suitably normalised) from the original raw data.

For this, we first define some helper functions.

In [6]:
parse_date = lambda x: (datetime.datetime.strptime(x.replace(' 00:00', ''), '%Y-%m-%d').date() - datetime.date(1995, 01, 01)).days

In [7]:
def get_latitude(postcode):
    try:
        return postcode_locations['lat'][postcode]
    except:
        return None

def get_longitude(postcode):
    try:
        return postcode_locations['long'][postcode]
    except:
        return None

In [8]:
data = pd.DataFrame()

data['transfer_date'] = property_transactions['transaction_transfer_date'].map(parse_date)
data['latitude'] = property_transactions['property_postcode'].map(get_latitude)
data['longitude'] = property_transactions['property_postcode'].map(get_longitude)
data = pd.concat([data, pd.get_dummies(property_transactions['property_type'])], axis=1)
data = pd.concat([data, pd.get_dummies(property_transactions['property_age'])], axis=1)
data = pd.concat([data, pd.get_dummies(property_transactions['property_duration'])], axis=1)
data['price'] = property_transactions['transaction_price']

In [9]:
data.describe()

,transfer_date,latitude,longitude,D,F,S,T,N,Y,F,L,U,price
count,2.185876e+07,2.183766e+07,2.183766e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07,2.185876e+07
mean,3.781426e+03,5.228254e+01,-1.273184e+00,2.320397e-01,1.807350e-01,2.786707e-01,3.085545e-01,8.981101e-01,1.018899e-01,7.655441e-01,2.344248e-01,3.110882e-05,1.709998e+05
std,2.233796e+03,1.128949e+00,1.309231e+00,4.221342e-01,3.847985e-01,4.483451e-01,4.618968e-01,3.025035e-01,3.025035e-01,4.236583e-01,4.236388e-01,5.577441e-03,2.017316e+05
min,0.000000e+00,4.989517e+01,-6.352647e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,1.955000e+03,5.144443e+01,-2.181589e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,7.400000e+04
50%,3.517000e+03,5.194465e+01,-1.275947e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.299500e+05
75%,5.556000e+03,5.328067e+01,-2.027250e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.090000e+05
max,8.093000e+03,1.000000e+02,1.758397e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.200000e+08


In [11]:
data.to_csv('../data/data-unnormalised.csv', index=False)